<a href="https://colab.research.google.com/github/karolyneehcs/MachineLearn/blob/master/%5BTCC%5D_SG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN3

By [Derrick Schultz](https://twitter.com/dvsch), with contributions from [crimeacs](https://twitter.com/EarthML1)

Just starting this...expect more updates soon.

If you find this helpful, please consider backing me on [Patreon](https://www.patreon.com/bustbright) or becoming a [YouTube channel member](https://www.youtube.com/channel/UCaZuPdmZ380SFUMKHVsv_AA/join).

## Setup

In [ ]:
!nvidia-smi -L

GPU 0: A100-SXM4-40GB (UUID: GPU-43224d89-6ee1-70b1-cd8e-eb1d69fa24d5)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Uninstall new JAX
!pip uninstall jax jaxlib -y
#GPU frontend
!pip install "jax[cuda11_cudnn805]==0.3.10" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
#CPU frontend
#!pip install jax[cpu]==0.3.10
#Downgrade Pytorch
!pip uninstall torch torchvision -y
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install timm==0.4.12 ftfy==6.1.1 ninja==1.10.2 imageio-ffmpeg opensimplex

Found existing installation: jax 0.3.21
Uninstalling jax-0.3.21:
  Successfully uninstalled jax-0.3.21
Found existing installation: jaxlib 0.3.20+cuda11.cudnn805
Uninstalling jaxlib-0.3.20+cuda11.cudnn805:
  Successfully uninstalled jaxlib-0.3.20+cuda11.cudnn805
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     |████████████████████████████████| 939 kB 4.6 MB/s 
     |████████████████████████████████| 175.7 MB 7.1 kB/s 
  Created wheel for jax: filename=jax-0.3.10-py3-none-any.whl size=1088066 sha256=a8d14052a1c004bc1e1d16fdcf9c4f128be68321a75e496d682343869b767975
  Stored in directory: /root/.cache/pip/wheels/1c/05/23/36730377cd7311156f1e5eb5e7c683d5cdac1654658c095cc5
Successfully built jax
Found existing installation: torch 1.12.1+cu113
Uninstalling torch-1.12.1+cu113:
  Successfully uninstalled torch-1.12.1+cu113
Found existing installation: torc

In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg3"):
    %cd "/content/drive/MyDrive/colab-sg3/stylegan3/"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg3
    %cd colab-sg3
    !git clone https://github.com/dvschultz/stylegan3
    %cd stylegan3
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan3
    %cd stylegan3
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install opensimplex

/content/drive/MyDrive
/content/drive/MyDrive/colab-sg3
Cloning into 'stylegan3'...
remote: Enumerating objects: 209, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 209 (delta 0), reused 2 (delta 0), pack-reused 206
Receiving objects: 100% (209/209), 4.23 MiB | 16.54 MiB/s, done.
Resolving deltas: 100% (86/86), done.
/content/drive/MyDrive/colab-sg3/stylegan3
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/gdown/cli.py", line 166, in main
    resume=args.continue_,
  File "/usr/local/lib/python3.7/dist-packages/gdown/download.py", line 202, in download
    for file in os.listdir(osp.dirname(output) 

This cell will update to the latest repo. Git and Drive/Colab don’t play as nicely as I’d like so 🤞. The other option is to delete your folder in Drive (after saving out `/results` and `/datasets`!) and running the script above to replace the entire folder.

In [ ]:
%cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- train.py gen_images.py gen_video.py README.md training/training_loop.py

[Errno 2] No such file or directory: '/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch'
/content/drive/MyDrive/colab-sg3/stylegan3
Already up to date.
No local changes to save


## Convert/Create Dataset
Pass a folder of images (just .pngs? TK) to create a zip file.

In [ ]:
!python dataset_tool.py --source=/content/drive/MyDrive/Dataset/Ghibli --dest=./datasets/ghibli-1024.zip

 36% 367/1028 [00:37<00:54, 12.04it/s]Error: Image 00000/img00000368.png attributes must be equal across all images of the dataset.  Got:
  dataset width/cur image width: 1024/1092
  dataset height/cur image height: 1024/1024
  dataset channels/cur image channels: 3/3
 36% 368/1028 [00:37<01:07,  9.76it/s]


## Training

Before you start training, read [this](https://github.com/dvschultz/stylegan3/blob/main/docs/configs.md).

Working Notes:
- It appears that you must use an SG3 pre-trained model for transfer learning. I _think_ you also want to match config to the pretrained model (`t` with `t`, `r` with `r`).
- For an `A100` I’ve found you can use a `--batch-gpu=8`. For other GPUs I recommend `--batch-gpu=4`.
- I see `~205 sec/kimg` on A100s, and `~325 sec/kimg` on V100s (1024, `r` config). This seems slightly slower than what [NVIDIA reports.](https://github.com/dvschultz/stylegan3/blob/main/docs/configs.md)

In [ ]:
!python train.py --help

Usage: train.py [OPTIONS]

  Train a GAN using the techniques described in
  the paper "Alias-Free Generative Adversarial
  Networks".

  Examples:

  # Train StyleGAN3-T for AFHQv2 using 8 GPUs.
  python train.py --outdir=~/training-runs --cfg=stylegan3-t --data=~/datasets/afhqv2-512x512.zip \
      --gpus=8 --batch=32 --gamma=8.2 --mirror=1

  # Fine-tune StyleGAN3-R for MetFaces-U using 1 GPU, starting from the pre-trained FFHQ-U pickle.
  python train.py --outdir=~/training-runs --cfg=stylegan3-r --data=~/datasets/metfacesu-1024x1024.zip \
      --gpus=8 --batch=32 --gamma=6.6 --mirror=1 --kimg=5000 --snap=5 \
      --resume=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-1024x1024.pkl

  # Train StyleGAN2 for FFHQ at 1024x1024 resolution using 8 GPUs.
  python train.py --outdir=~/training-runs --cfg=stylegan2 --data=~/datasets/ffhq-1024x1024.zip \
      --gpus=8 --batch=32 --gamma=10 --mirror=1 --aug=noaug

Options:
  --outdir DIR 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!python train.py --outdir=./results --cfg=stylegan3-t --data=/content/drive/MyDrive/colab-sg3/stylegan3/datasets/ghibli-1024.zip \
--gpus=1 --batch=32 --batch-gpu=8 --gamma=10.0 --mirror=1 --kimg=25000 --snap=1 \
--resume=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-ffhq-1024x1024.pkl

# 12.29 (A100 * 2500 kimg * 1.05 = tempo estimado de treinamento )


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 32768,
    "channel_max": 512,
    "magnitude_ema_beta": 0.9988915792636801
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 10.0,
    "blur_init_sigma": 0
  },
  "data_loader_kwargs": {
    "pin_memory

## Image Generation

In [ ]:
!python gen_images.py --help

In [ ]:
!python gen_images.py --outdir=out --trunc=1 --seeds=2 \
    --network=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-afhqv2-512x512.pkl

## Video Generation


In [ ]:
!python gen_video.py --help


In [ ]:
!python gen_video.py --output=/content/lerp.mp4 --trunc=1 --seeds=100-124 --grid=1x1 --w-frames=72 \
    --network=/content/drive/MyDrive/colab-sg3/stylegan3/results/00014-stylegan3-r-drawn-gems-1024-gpus1-batch32-gamma10/network-snapshot-000104.pkl